In [1]:
import os
# from apikey import apikey

import streamlit as st
from langchain import OpenAI 
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma 
#from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient


os.environ["OPENAI_API_KEY"] = ''

def clear_history():
    if 'history' in st.session_state:
        del st.session_state['history']

st.title('Chat with Document')

def get_vector_store():

    client = qdrant_client.QdrantClient(
        os.getenv("QDRANT_HOST"),
        api_key=os.getenv("QDRANT_API_KEY")
    )

    embeddings = OpenAIEmbeddings()

    vector_store = Qdrant(
        client=client, 
        collection_name=os.getenv("QDRANT_COLLECTION_NAME"), 
        embeddings=embeddings,
    )

    return vector_store


     # create vector store
    vector_store = get_vector_store()
    retriever = vector_store.as_retriever()
    
    # create chain 
    # qa = RetrievalQA.from_chain_type(
    #     llm=OpenAI(),
    #     chain_type="stuff",
    #     retriever=vector_store.as_retriever()
    # )
    
    #llm = OpenAI(temperature=0)
    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

    #chain = RetrievalQA.from_chain_type(llm, retriever=retriever )
    crc = ConversationalRetrievalChain.from_llm(llm,retriever)
    st.session_state.crc = crc
    st.success('Database succesfully retrieved.')

question = st.text_input('Input your question')

if question:
    if 'crc' in st.session_state:
        crc = st.session_state.crc
        if 'history' not in st.session_state:
            st.session_state['history'] = []

        response = crc.run({'question':question,'chat_history':st.session_state['history']})

        st.session_state['history'].append((question,response))
        st.write(response)

        #st.write(st.session_state['history'])
        for prompts in st.session_state['history']:
            st.write("Question: " + prompts[0])
            st.write("Answer: " + prompts[1])    

2024-04-05 23:45:33.663 
  command:

    streamlit run c:\Users\derek\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
